### Imports

In [52]:
import geopandas as gpd
from shapely.geometry import Point, box
import rasterio
from rasterio.transform import from_origin, Affine
from rasterio.features import rasterize
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import json
import time
import math
start = time.time()
def get_elapsed_time():
    global start
    elapsed = time.time() - start
    start = time.time()
    return elapsed

### Bounding Box

In [53]:
def create_bounding_box(x,y,buffer_distance):
    """
    Create a bounding box around a point with a buffer distance in meters
    """
    # Define the bounding box for a 10-mile radius around Hancock Hill in degrees
    buffer_distance_meters = buffer_distance
    buffer_distance_lat = buffer_distance_meters / 111000
    buffer_distance_lon = buffer_distance_meters / (111000 * math.cos(math.radians(y)))
    minx = x - buffer_distance_lon
    maxx = x + buffer_distance_lon
    miny = y - buffer_distance_lat
    maxy = y + buffer_distance_lat

    return box(minx, miny, maxx, maxy)

### Load Data

In [61]:
def load_soil_data(bounding_box):
    spatial_layer = 'mupolygon'
    
    spatial_gdf = gpd.read_file(soildb_path, layer=spatial_layer)
    print("Read spatial data form file {}".format(get_elapsed_time()))


    # Connect to the SQLite database and load relevant tables
    conn = sqlite3.connect(soildb_path)
    component_df = pd.read_sql_query("SELECT mukey, taxorder, map_l, map_r, map_h, airtempa_l, airtempa_r, airtempa_h FROM component", conn)
    conn.close()
    print("Read tabular data from database {}".format(get_elapsed_time()))

    # Merge the tabular data with the spatial data on 'mukey'
    merged_gdf = spatial_gdf.merge(component_df, on='mukey')
    print("Merged tabular and spatial data {}".format(get_elapsed_time()))
    
    bounding_box_gdf = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs=main_crs)
    bounding_box_gdf = bounding_box_gdf.to_crs(merged_gdf.crs)
    # bounding_box = bounding_box_gdf.geometry.iloc[0]

    # Filter the merged GeoDataFrame for the area of interest
    merged_gdf = merged_gdf[merged_gdf.geometry.intersects(bounding_box_gdf)]
    print("Filtered data for bounding box {}".format(get_elapsed_time()))

    return merged_gdf

In [50]:
def load_dem_data(bounding_box):
    with rasterio.open(combined_dem_path) as src:
        dem_crs = src.crs
        dem_transform = src.transform
        bbox_gdf = gpd.GeoDataFrame({"geometry": [bounding_box]}, crs=main_crs)

        bbox_gdf = bbox_gdf.to_crs(dem_crs)
        bbox = bbox_gdf.geometry.iloc[0].bounds
        
        dem_bounds = src.bounds
        if (bbox[0] < dem_bounds.left or bbox[2] > dem_bounds.right or
            bbox[1] < dem_bounds.bottom or bbox[3] > dem_bounds.top):
            raise ValueError("Bounding box is out of DEM bounds.")
        window = rasterio.windows.from_bounds(*bbox, transform=dem_transform)
        dem_clip = src.read(1, window=window)
        transform_clip = src.window_transform(window)

        
    # Calculate the dimensions of the target raster
    width = int((bbox[2] - bbox[0]) / resolution)
    height = int((bbox[3] - bbox[1]) / resolution)
    print("Loaded DEM data {}".format(get_elapsed_time()))

    # Check if dimensions are valid
    if width <= 0 or height <= 0:
        raise ValueError(f"Invalid target dimensions: width={width}, height={height}")

    # Create an array to hold the resampled DEM data
    dem_data = np.empty((height, width), dtype=np.float32)

    # Calculate the transform for the target raster
    dst_transform = from_origin(bbox[0], bbox[3], resolution, -resolution)

    reproject(
        source=dem_clip,
        destination=dem_data,
        src_transform=transform_clip,
        src_crs=dem_crs,
        dst_transform=dst_transform,
        dst_crs=main_crs,
        resampling=Resampling.bilinear
    )
    print("Resampled DEM data {}".format(get_elapsed_time()))
    return dem_data


In [58]:
def load_water_data(bounding_box):
    # Path to your GPKG file

    # Define the correct layer names after listing them
    flowlines_layer = 'nhdflowline_or'  # Replace with the actual flowline layer name
    waterbodies_layer = 'nhdwaterbody_or'  # Replace with the actual waterbody layer name
    flowline_vaa_layer = 'nhdplusflowlinevaa_or'  # Replace with the actual VAA layer name
    eromma_layer = 'nhdpluseromma_or'  # Replace with the actual EROMMA layer name

    # Define the stream order column, mean annual flow column, and buffer sizes
    print("Processing Water Data...")
    # Load the Flowlines, Waterbodies, and value-added attributes layers from the GPKG
    flowlines_gdf = gpd.read_file(gpkg_path, layer=flowlines_layer)
    waterbodies_gdf = gpd.read_file(gpkg_path, layer=waterbodies_layer)
    eromma_df = gpd.read_file(gpkg_path, layer=eromma_layer)
    print("Read data from file {}".format(get_elapsed_time()))
    
    proj_bounding_box = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs=main_crs)
    proj_bounding_box = proj_bounding_box.to_crs(flowlines_gdf.crs)
    
    eromma_df = eromma_df[['nhdplusid', 'qa']]

    flowlines_gdf = gpd.overlay(flowlines_gdf, proj_bounding_box, how='intersection')
    waterbodies_gdf = gpd.overlay(waterbodies_gdf, proj_bounding_box, how='intersection')
    print("Filtered data for bounding box {}".format(get_elapsed_time()))

    flowlines_gdf = flowlines_gdf.merge(eromma_df, on='nhdplusid')
    print("Merged data {}".format(get_elapsed_time()))


    # Buffer the flowlines by mean annual flow
    flowlines_gdf['buffer'] = .001 #flowlines_gdf[mean_annual_flow_col].apply(lambda x: np.sqrt(x) / 100)  # Adjust the multiplier as needed
    buffered_flowlines_gdf = flowlines_gdf.copy()
    buffered_flowlines_gdf['geometry'] = flowlines_gdf.buffer(buffered_flowlines_gdf['buffer'])
    print("Buffered Flowlines {}".format(get_elapsed_time()))

    # Combine the buffered flowlines with the waterbodies polygons
    combined_gdf = gpd.GeoDataFrame(pd.concat([buffered_flowlines_gdf, waterbodies_gdf], ignore_index=True))
    print("Combined Flowlines and Waterbodies {}".format(get_elapsed_time()))


    return combined_gdf

### Rasterize Functions

In [33]:
def normalize_raster(raster_data):
    raster_min = np.nanmin(raster_data)
    raster_max = np.nanmax(raster_data)
    raster_data_normalized = 255 * (raster_data - raster_min) / (raster_max - raster_min)
    raster_data_normalized = raster_data_normalized.astype(np.uint8)
    return raster_data_normalized

In [34]:
def print_raster(raster_data, title):
    normalized_raster = normalize_raster(raster_data)
    plt.imsave('{}.png'.format(title), normalized_raster, cmap='gray', vmin=0, vmax=255)
    print("Successfully created raster: {}".format(title))

In [35]:
def rasterize_layer(gdf, layer_name, attribute_mapping = None):
    xmin, ymin, xmax, ymax = gdf.total_bounds
    width = int((xmax - xmin) / resolution)
    height = int((ymax - ymin) / resolution)
    transform = from_origin(xmin, ymax, resolution, resolution)
    raster_data = np.full((height, width), np.nan)
    # Rasterize the clay content
    for _, row in gdf.iterrows():
        
        value = row[layer_name]
        
        if attribute_mapping:
            value = attribute_mapping[value]    

        shapes = [(row['geometry'], value)]

        burned = rasterize(
            shapes=shapes,
            out_shape=(height, width),
            fill=np.nan,
            transform=transform,
            all_touched=True,
            dtype='float32'
        )
        raster_data = np.where(np.isnan(raster_data), burned, raster_data)

    return raster_data

### Primary Execution Script

In [62]:
start = time.time()
start_time = start
with open('constants.json') as f:
    constants = json.load(f)['constants']

## Primary Layer Data
soildb_path = 'SSURGODB.gpkg' 
combined_dem_path = 'merged_dem.tif'
gpkg_path = 'nhdplus_epasnapshot2022_or.gpkg'
main_crs = 'EPSG:4326'

## Focus Area
resolution = 15 / 111000
lat = 45.798
long = -118.932
width = 2 * 1609.34

bounding_box = create_bounding_box(long,lat,width)  # 5 miles in meters


start_time = time.time()    
soils_gdf = load_soil_data(bounding_box)
print("Soil data loaded")
water_gdf = load_water_data(bounding_box)
print("Water data loaded")
## DEM (already rasterized)
dem_raster = load_dem_data(bounding_box)
print("DEM data loaded")


print(" ================================")
print(soils_gdf.bounds)
print(" ------------------------------")

water_gdf = water_gdf.to_crs(soils_gdf.crs)
print(water_gdf.bounds)
print(" ================================")

precip_raster = rasterize_layer(soils_gdf, 'map_r')
temp_raster = rasterize_layer(soils_gdf, 'airtempa_r')
soil_raster = rasterize_layer(soils_gdf, 'taxorder', constants['soil_type_mapping'])
water_raster = rasterize_layer(water_gdf, 'qa')
print("Rasterization complete")

print_raster(dem_raster, 'dem')
print_raster(precip_raster, 'precip')
print_raster(temp_raster, 'temp')
print_raster(soil_raster, 'soil')
print_raster(water_raster, 'water')
print ("Total time: {}".format(time.time() - start_time))


Read spatial data form file 8.561441898345947
Read tabular data from database 0.32973265647888184
Merged tabular and spatial data 0.03200125694274902
Filtered data for bounding box 0.5395748615264893
Soil data loaded
Processing Water Data...
Read data from file 56.669527530670166
Filtered data for bounding box 7.26214075088501
Merged data 0.2815406322479248
Buffered Flowlines 0.04656076431274414
Combined Flowlines and Waterbodies 0.0019989013671875
Water data loaded


C:\Users\schro\AppData\Local\Temp\ipykernel_47752\3923642483.py:34: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffered_flowlines_gdf['geometry'] = flowlines_gdf.buffer(buffered_flowlines_gdf['buffer'])


Loaded DEM data 0.6525444984436035
Resampled DEM data 0.007999181747436523
DEM data loaded
             minx       miny        maxx       maxy
26327 -118.981560  45.592143 -118.378738  45.969016
26328 -118.970401  45.525173 -118.379419  45.976782
26329 -118.983568  45.572468 -118.376660  45.978204
26331 -118.974966  45.739509 -118.405261  45.955727
26415 -119.138808  45.351568 -118.314069  46.000916
26416 -119.138808  45.351568 -118.314069  46.000916
26425 -119.434645  45.526714 -118.634111  45.995478
26448 -119.327736  45.604910 -118.566194  46.000980
26462 -119.248260  45.515957 -118.390996  46.000954
26485 -118.953692  45.655094 -118.400503  45.932595
26506 -119.071462  45.715986 -118.808173  46.000759
26510 -119.434574  45.602163 -118.533383  46.000975
26511 -119.434635  45.596602 -118.531205  46.000984
26512 -119.434653  45.603829 -118.503334  46.001043
26513 -119.434653  45.603829 -118.503334  46.001043
 ------------------------------
           minx       miny        maxx       

C:\Users\schro\AppData\Local\Temp\ipykernel_47752\3347337340.py:4: RuntimeWarning: invalid value encountered in divide
  raster_data_normalized = 255 * (raster_data - raster_min) / (raster_max - raster_min)
C:\Users\schro\AppData\Local\Temp\ipykernel_47752\3347337340.py:5: RuntimeWarning: invalid value encountered in cast
  raster_data_normalized = raster_data_normalized.astype(np.uint8)


Successfully created raster: precip
Successfully created raster: temp
Successfully created raster: soil
Successfully created raster: water
Total time: 100.54794430732727


In [57]:
bounding_box.bounds

(-118.97359140524283,
 45.76900288288289,
 -118.89040859475718,
 45.826997117117116)